In [ ]:
# Install the OpenSlide C library and Python bindings
!apt-get install openslide-tools
!pip install openslide-python
!pip install tifffile

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from openslide import open_slide, __library_version__ as openslide_version, OpenSlide
import os
from PIL import Image
from skimage.color import rgb2gray
import math

In [ ]:
from google.colab import drive
drive.mount('/content/drive/',  force_remount=True)

In [ ]:
### Reading slides and mask ###
slide_num = '110'

slide_path = '/content/drive/My Drive/tumor/tumor_' + slide_num + '.tif'
tumor_mask_path = '/content/drive/My Drive/cancer_detection/backup/annotation_mask/teste7/tumor_' + slide_num +'_mask.tif'

slide = open_slide(slide_path)
tumor_mask = open_slide(tumor_mask_path)

print ("Reading WSI from %s | width: %d, height: %d" % (slide_path, 
                                                        slide.level_dimensions[0][0], 
                                                        slide.level_dimensions[0][1]))

print ("Reading annotation mask from %s | width: %d, height: %d" % (tumor_mask_path, 
                                                        tumor_mask.level_dimensions[0][0], 
                                                        tumor_mask.level_dimensions[0][1]))

In [ ]:
def read_slide(slide, x, y, level, width, height, as_float=False):
    im = slide.read_region((x,y), level, (width, height))
    im = im.convert('RGB') # drop the alpha channel
    if as_float:
        im = np.asarray(im, dtype=np.float32)
    else:
        im = np.asarray(im)
    assert im.shape == (height, width, 3)
    return im
    
slide_image = read_slide(slide, 
                         x=0, 
                         y=0, 
                         level=5, 
                         width=slide.level_dimensions[5][0], 
                         height=slide.level_dimensions[5][1])

mask_image = read_slide(tumor_mask, 
                         x=0, 
                         y=0, 
                         level=5, 
                         width=slide.level_dimensions[5][0], 
                         height=slide.level_dimensions[5][1])
#show figs
#plt.figure(figsize=(10,10), dpi=100)
#plt.imshow(slide_image)

#plt.figure(figsize=(10,10), dpi=100)
#plt.imshow(mask_image)

FILTERS

In [ ]:
# From "A method for normalizing histology slides for quantitative analysis" - M Macenko, M Niethammer, JS Marron, D Borland, JT Woosley, G Xiaojun, C Schmitt, NE Thomas, IEEE ISBI, 2009. dx.doi.org/10.1109/ISBI.2009.5193250
# Stain normalization method for H&E stained images
def normalizeStaining(img, saveFile=None, Io=240, alpha=1, beta=0.15):
             
    HERef = np.array([[0.5626, 0.2159],
                      [0.7201, 0.8012],
                      [0.4062, 0.5581]])
        
    maxCRef = np.array([1.9705, 1.0308])
    
    # define height and width of image
    h, w, c = img.shape
    
    # reshape image
    img = img.reshape((-1,3))

    # calculate optical density
    OD = -np.log((img.astype(np.float)+1)/Io)
    
    # remove transparent pixels
    ODhat = OD[~np.any(OD<beta, axis=1)]
        
    # compute eigenvectors
    eigvals, eigvecs = np.linalg.eigh(np.cov(ODhat.T))
        
    # project on the plane spanned by the eigenvectors corresponding to the two 
    # largest eigenvalues    
    That = ODhat.dot(eigvecs[:,1:3])
    
    phi = np.arctan2(That[:,1],That[:,0])
    
    minPhi = np.percentile(phi, alpha)
    maxPhi = np.percentile(phi, 100-alpha)
    
    vMin = eigvecs[:,1:3].dot(np.array([(np.cos(minPhi), np.sin(minPhi))]).T)
    vMax = eigvecs[:,1:3].dot(np.array([(np.cos(maxPhi), np.sin(maxPhi))]).T)
    
    # a heuristic to make the vector corresponding to hematoxylin first and the 
    # one corresponding to eosin second
    if vMin[0] > vMax[0]:
        HE = np.array((vMin[:,0], vMax[:,0])).T
    else:
        HE = np.array((vMax[:,0], vMin[:,0])).T
    
    # rows correspond to channels (RGB), columns to OD values
    Y = np.reshape(OD, (-1, 3)).T
    
    # determine concentrations of the individual stains
    C = np.linalg.lstsq(HE,Y, rcond=None)[0]
    
    # normalize stain concentrations
    maxC = np.array([np.percentile(C[0,:], 99), np.percentile(C[1,:],99)])
    tmp = np.divide(maxC,maxCRef)
    C2 = np.divide(C,tmp[:, np.newaxis])
    
    # recreate the image using reference mixing matrix
    Inorm = np.multiply(Io, np.exp(-HERef.dot(C2)))
    Inorm[Inorm>255] = 254
    plt.figure(figsize=(10,10), dpi=100)
    Inorm = np.reshape(Inorm.T, (h, w, 3)).astype(np.uint8)  
        
    return Image.fromarray(Inorm)  

#stain normalization
nomr_img = normalizeStaining(img = slide_image)

In [ ]:
# Removing backround
# I am not getting the resulting image from the stain normalization method, I am applying the filter to remove the background on the original image and creating a mask
def grays_filter(rgb, tolerance=15):
  
  #Mask to filter out pixels where the red, green, and blue channel values are similar.
  '''
  Filtro para a área de sombra no slide (área de sombra = gradiente de tons de cinza escuro a claro). 
  Um pixel cinza tem valores de canal vermelho, verde e azul que estão próximos. 
  A função filter_grays () filtra os pixels que têm valores de vermelho, azul e verde que estão dentro de uma certa tolerância um do outro. 
  A tolerância padrão é 15. O filtro cinzas também filtra pixels brancos e pretos, pois eles têm valores semelhantes de vermelho, verde e azul.
  '''
  rgb = rgb.astype(np.int)
  rg_diff = abs(rgb[:, :, 0] - rgb[:, :, 1]) <= tolerance
  rb_diff = abs(rgb[:, :, 0] - rgb[:, :, 2]) <= tolerance
  gb_diff = abs(rgb[:, :, 1] - rgb[:, :, 2]) <= tolerance
  result = ~(rg_diff & rb_diff & gb_diff)

  return result

#background removal
rgb = slide_image
mask = grays_filter(rgb) # the mask is important, the background_removal_img is just to plot
background_removal_img = rgb * np.dstack([mask, mask, mask])

In [ ]:
#show figs
plt.figure(figsize=(10,10), dpi=100)
plt.imshow(nomr_img)

plt.figure(figsize=(10,10), dpi=100)
plt.imshow(background_removal_img)

In [ ]:
### All filters together ###
#after remove aplling mask to remove the background
img_result = nomr_img * np.dstack([mask, mask, mask])

#show figs
#plt.figure(figsize=(5,5), dpi=100)
#plt.imshow(img_result)

In [ ]:
# making the image and the mask square to obtain patches with 256x256 (it need to be improvement)
def square_image(img):  
    #Getting the bigger side of the image
    s = max(img.shape[0:2])

    #Creating a dark square with NUMPY  
    f = np.zeros((s,s,3),np.uint8)

    #Getting the centering position
    ax,ay = (s - img.shape[1])//2,(s - img.shape[0])//2

    #Pasting the 'image' in a centering position
    f[ay:img.shape[0]+ay,ax:ax+img.shape[1]] = img    
    return f

img_result_square = square_image(img_result)
mask_image_square = square_image(mask_image)

#show image
#plt.figure(figsize=(10,10), dpi=100)
#plt.imshow(img_result_square)

#plt.figure(figsize=(10,10), dpi=100)
#plt.imshow(mask_image_square)

In [ ]:
#plotting the mask on the filtered image
plt.figure(figsize=(10,10), dpi=100)
plt.imshow(img_result_square)
plt.imshow(mask_image_square, cmap='viridis', alpha=0.44)

CLASSIFICATION REGIONS

In [ ]:
#Base directory

train_dir = '/content/drive/My Drive/cancer_detection/paper/patch_train'
val_dir = '/content/drive/My Drive/cancer_detection/paper/patch_val'
test_dir = '/content/drive/My Drive/cancer_detection/paper/patch_test'

In [ ]:
## Building train and validation pacthes

img = img_result_square
mask = mask_image_square

patch_size = 256

img_shape = img.shape
mask_shape = mask.shape

tile_size = (patch_size, patch_size)
offset = (patch_size, patch_size)

i_range = (int(math.ceil(img_shape[0]/(offset[1] * 1.0))))
j_range= (int(math.ceil(img_shape[1]/(offset[0] * 1.0))))

i=0
j=0
count_patch = 0
count_zero = 0

for i in range(i_range):
  for j in range(j_range):
    cropped_img = img[offset[1]*i:min(offset[1]*i+tile_size[1], img_shape[0]), offset[0]*j:min(offset[0]*j+tile_size[0], img_shape[1])]
    cropped_img_mask = mask[offset[1]*i:min(offset[1]*i+tile_size[1], mask_shape[0]), offset[0]*j:min(offset[0]*j+tile_size[0], mask_shape[1])]
    cont_zero_img = np.count_nonzero(cropped_img)
    is_all_zero = np.all((cropped_img == 0)) # if the array contains just zero or count_zero is more than 50% of the image the patch is does considered. ---> DISCUSSING POINT <---
    if is_all_zero or cont_zero_img < 32768: # -> 32768 is 50% from a 256x256 patch
      count_zero += 1
    else:
      non_zero_img = np.count_nonzero(cropped_img)
      non_zero_mask = np.count_nonzero(cropped_img_mask)
      percent = (non_zero_mask/non_zero_img)*100
      if count_patch % 5 == 0: # for each 5 patchs of train save one as validation 
        drive_path = val_dir
        if percent > 10: # if the mask occupies a region greater than 10% of the non-zero pixel (tissues) the patch is classified as positive ---> DISCUSSING POINT <---
          print('save' + slide_num + '_' + str(i) + '_' + str(j) + ': class 1')
          plt.imsave(drive_path + '/1/' + slide_num + '_' + str(i) + '_' + str(j) + '.jpg', cropped_img)
        else:
          print('save' + slide_num + '_' + str(i) + '_' + str(j) + ': class 0')
          plt.imsave(drive_path + '/0/' + slide_num + '_' + str(i) + '_' + str(j) + '.jpg', cropped_img)
      else:
        drive_path = train_dir
        if percent > 10: 
          print('save' + slide_num + '_' + str(i) + '_' + str(j) + ': class 1')
          plt.imsave(drive_path + '/1/' + slide_num + '_' + str(i) + '_' + str(j) + '.jpg', cropped_img)
        else:
          print('save' + slide_num + '_' + str(i) + '_' + str(j) + ': class 0')
          plt.imsave(drive_path + '/0/' + slide_num + '_' + str(i) + '_' + str(j) + '.jpg', cropped_img)
      count_patch +=1
    j +=1
  i +=1
print('count_zero', count_zero)
print('count_patch', count_patch)

In [ ]:
## Building test pacthes
img = img_result_square
mask = mask_image_square

patch_size = 256

img_shape = img.shape
mask_shape = mask.shape

tile_size = (patch_size, patch_size)
offset = (patch_size, patch_size)

i_range = (int(math.ceil(img_shape[0]/(offset[1] * 1.0))))
j_range= (int(math.ceil(img_shape[1]/(offset[0] * 1.0))))

i=0
j=0
count_zero = 0
drive_path = test_dir

for i in range(i_range):
  for j in range(j_range):
    cropped_img = img[offset[1]*i:min(offset[1]*i+tile_size[1], img_shape[0]), offset[0]*j:min(offset[0]*j+tile_size[0], img_shape[1])]
    cropped_img_mask = mask[offset[1]*i:min(offset[1]*i+tile_size[1], mask_shape[0]), offset[0]*j:min(offset[0]*j+tile_size[0], mask_shape[1])]
    cont_zero_img = np.count_nonzero(cropped_img)
    is_all_zero = np.all((cropped_img == 0)) # if the array contains just zero or count_zero is more than 50% of the image the patch is does considered. ---> DISCUSSING POINT <---
    if is_all_zero or cont_zero_img < 32768: # -> 32768 is 50% from a 256x256 patch
      count_zero += 1
    else:
      non_zero_img = np.count_nonzero(cropped_img)
      non_zero_mask = np.count_nonzero(cropped_img_mask)
      percent = (non_zero_mask/non_zero_img)*100
      if percent > 10: # if the mask occupies a region greater than 10% of the non-zero pixel (tissues) the patch is classified as positive ---> DISCUSSING POINT <---
        print('save' + slide_num + '_' + str(i) + '_' + str(j) + ': class 1')
        plt.imsave(drive_path + '/1/' + slide_num + '_' + str(i) + '_' + str(j) + '.jpg', cropped_img)
      else:
        print('save' + slide_num + '_' + str(i) + '_' + str(j) + ': class 0')
        plt.imsave(drive_path + '/0/' + slide_num + '_' + str(i) + '_' + str(j) + '.jpg', cropped_img)
      if percent > 10: 
        print('save' + slide_num + '_' + str(i) + '_' + str(j) + ': class 1')
        plt.imsave(drive_path + '/1/' + slide_num + '_' + str(i) + '_' + str(j) + '.jpg', cropped_img)
      else:
        print('save' + slide_num + '_' + str(i) + '_' + str(j) + ': class 0')
        plt.imsave(drive_path + '/0/' + slide_num + '_' + str(i) + '_' + str(j) + '.jpg', cropped_img)
    j +=1
  i +=1
print('count_zero', count_zero)